In [5]:
import flair.datasets
corpus = flair.datasets.UD_INDONESIAN()

2023-10-31 20:37:37,927 Reading data from C:\Users\User\.flair\datasets\ud_indonesian
2023-10-31 20:37:37,927 Train: C:\Users\User\.flair\datasets\ud_indonesian\id_gsd-ud-train.conllu
2023-10-31 20:37:37,927 Dev: C:\Users\User\.flair\datasets\ud_indonesian\id_gsd-ud-dev.conllu
2023-10-31 20:37:37,941 Test: C:\Users\User\.flair\datasets\ud_indonesian\id_gsd-ud-test.conllu
2023-10-31 20:37:44,246 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-10-31 20:37:44,646 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-10-31 20:37:45,063 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [2]:
tag_type = 'upos'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

C:\Users\User\AppData\Local\Temp\ipykernel_10872\3418903033.py:2: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


In [6]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings
from typing import List
embedding_types: List[TokenEmbeddings] = [
 WordEmbeddings('id-crawl'),
 WordEmbeddings('id'),
]
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [7]:
from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                      tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

2023-10-31 20:44:56,935 SequenceTagger predicts: Dictionary with 20 tags: O, PROPN, AUX, DET, NOUN, PRON, VERB, ADP, PUNCT, ADV, CCONJ, SCONJ, NUM, ADJ, PART, SYM, INTJ, X, <START>, <STOP>


In [ ]:
from flair.trainers import ModelTrainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-universal-pos',
 learning_rate=0.1,
 mini_batch_size=32,
 max_epochs=10)

In [8]:
from flair.data import Sentence
sentence = Sentence('Pada awal pidatonya, Yudhoyono para peserta untuk mengheningkan cipta sejenak bagi korban gempa dan Tsunami di Nanggroe Aceh Darussalam dan Nias, Sumatra Utara.')

In [9]:
sentence = sentence
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

2023-10-31 20:45:43,034 SequenceTagger predicts: Dictionary with 20 tags: O, PROPN, AUX, DET, NOUN, PRON, VERB, ADP, PUNCT, ADV, CCONJ, SCONJ, NUM, ADJ, PART, SYM, INTJ, X, <START>, <STOP>
Sentence[26]: "Pada awal pidatonya, Yudhoyono para peserta untuk mengheningkan cipta sejenak bagi korban gempa dan Tsunami di Nanggroe Aceh Darussalam dan Nias, Sumatra Utara." → ["Pada"/ADP, "awal"/NOUN, "pidatonya"/NOUN, ","/PUNCT, "Yudhoyono"/PROPN, "para"/DET, "peserta"/NOUN, "untuk"/SCONJ, "mengheningkan"/VERB, "cipta"/NOUN, "sejenak"/ADJ, "bagi"/ADP, "korban"/NOUN, "gempa"/NOUN, "dan"/CCONJ, "Tsunami"/PROPN, "di"/ADP, "Nanggroe"/PROPN, "Aceh"/PROPN, "Darussalam"/PROPN, "dan"/CCONJ, "Nias"/PROPN, ","/PUNCT, "Sumatra"/PROPN, "Utara"/PROPN, "."/PUNCT]


In [14]:
print(len(sentence))

26


In [12]:
print(sentence[1].tag)

NOUN


In [13]:
print(sentence.get_each_embedding)

<bound method DataPoint.get_each_embedding of Sentence[3]: "aku makan nasi" → ["aku"/PRON, "makan"/VERB, "nasi"/NOUN]>


In [19]:
print(type(sentence[2].tag))

<class 'str'>


### Augmentasi

In [13]:
dataset_canonical_dir  = '../Dataset/canonical/train/'

### batas

In [24]:
import datasets
from datasets import load_dataset
from datasets import load_from_disk
import re

In [22]:
raw_dataset = load_dataset("id_liputan6", 'canonical', data_dir="../Dataset/")

In [ ]:
'Presiden Susilo Bambang Yudhoyono memusatkan bahwa tantangan terbesar yang dihadapi bangsa-bangsa Asia dan Afrika jangka ini adalah kasus kesengsaraan yang sangat buruk .'

In [25]:
def data_cleaning(data):
    article = data['clean_article']

    # Remove starting 'Liputan6.com'
    
    if article.startswith("Liputan6.com,"):
        article = re.sub(r'Liputan6.com, [a-zA-z]*:', '', article).lstrip()

    # Remove author name at the end (pattern: (NAME))
    article = re.sub(r'\(\w+\/[\w\s]+ [\w\s]+\).', '', article).rstrip()

    # Store the cleansed article back into the example
    data['clean_article'] = article
    return data

In [26]:
dataset = raw_dataset.map(data_cleaning)

In [65]:
text = dataset['train'][0]['clean_article']
sentence = Sentence(text)
print(sentence)

Sentence[148]: "Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Syaratnya, penanganan penyelesaian konflik Maluku harus dimulai dari awal kerusuhan, yakni 19 Januari 1999. Demikian hasil Musyawarah Wilayah I PBB Maluku yang dimulai Sabtu pekan silam dan berakhir Senin (31/12) di Ambon. Menurut seorang fungsionaris PBB Ridwan Hasan, persoalan di Maluku bisa selesai asalkan pemerintah dan aparat keamanan serius menangani setiap persoalan di Maluku secara komprehensif dan bijaksana. Itulah sebabnya, PBB wilayah Maluku akan menjadikan penyelesaian konflik sebagai agenda utama partai. PBB Maluku juga akan mendukung penegakan hukum secara terpadu dan tanpa pandang bulu. Siapa saja yang melanggar hukum harus ditindak. Ridwan berharap, Ketua PBB Maluku yang baru, Ali Fauzi, dapat menindak lanjuti agenda politik partai yang telah diamanatkan dan mau mendukung penegakan hukum di Maluku."


In [66]:
print(type(sentence))

<class 'flair.data.Sentence'>


In [67]:
sentence = sentence
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

2023-10-24 13:36:16,429 SequenceTagger predicts: Dictionary with 20 tags: O, PROPN, AUX, DET, NOUN, PRON, VERB, ADP, PUNCT, ADV, CCONJ, SCONJ, NUM, ADJ, PART, SYM, INTJ, X, <START>, <STOP>
Sentence[148]: "Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Syaratnya, penanganan penyelesaian konflik Maluku harus dimulai dari awal kerusuhan, yakni 19 Januari 1999. Demikian hasil Musyawarah Wilayah I PBB Maluku yang dimulai Sabtu pekan silam dan berakhir Senin (31/12) di Ambon. Menurut seorang fungsionaris PBB Ridwan Hasan, persoalan di Maluku bisa selesai asalkan pemerintah dan aparat keamanan serius menangani setiap persoalan di Maluku secara komprehensif dan bijaksana. Itulah sebabnya, PBB wilayah Maluku akan menjadikan penyelesaian konflik sebagai agenda utama partai. PBB Maluku juga akan mendukung penegakan hukum secara terpadu dan tanpa pandang bulu. Siapa saja yang melanggar hukum harus ditindak. Ridwan berharap, Ketua PBB Ma

In [69]:
sentence = "Aku <mask> makan nasi"

### Tesaurus

In [1]:
from tesaurus import Tesaurus

te = Tesaurus()
te.cari("makan")
print(te)

AttributeError: 'NoneType' object has no attribute 'children'

### Tradisional

In [37]:
import os

In [38]:
def countFiles(directory):
    i = 0
    for file in os.scandir(directory):
        if file.is_file():
            i +=1
    return i

In [39]:
print(countFiles(dataset_canonical_dir))

193883


In [48]:
import json
def getArticle(filename):
    data = json.load(open(filename, "r"))
    article = data['clean_article']

    # Remove starting 'Liputan6.com'
    
    if article.startswith("Liputan6.com,"):
        article = re.sub(r'Liputan6.com, [a-zA-z]*:', '', article).lstrip()

    # Remove author name at the end (pattern: (NAME))
    article = re.sub(r'\(\w+\/[\w\s]+ [\w\s]+\).', '', article).rstrip()

    # Store the cleansed article back into the example
    data['clean_article'] = article
    return data

In [49]:
jml = 0
for file in os.scandir(dataset_canonical_dir):
    translated_article = []
    jml += 1
    file_name = os.path.splitext(file)[0] + ".json"

    if file.is_file():
        print(getArticle(file_name))

    if jml == 1:
        break

AttributeError: 'list' object has no attribute 'startswith'

In [42]:
def data_cleaning(data):
    article = data['clean_article']

    # Remove starting 'Liputan6.com'
    
    if article.startswith("Liputan6.com,"):
        article = re.sub(r'Liputan6.com, [a-zA-z]*:', '', article).lstrip()

    # Remove author name at the end (pattern: (NAME))
    article = re.sub(r'\(\w+\/[\w\s]+ [\w\s]+\).', '', article).rstrip()

    # Store the cleansed article back into the example
    data['clean_article'] = article
    return data